In [47]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [48]:
# Téléchargement des stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
data = pd.read_csv("movie_review.csv")
data.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


In [50]:
# Cleaning
import re

# Clean the 'text' column: removing punctuation and underscores
data["text"] = data["text"].apply(lambda x: re.sub(r'[^\w\s]|_', '', str(x)))


print(data)


       fold_id cv_tag  html_id  sent_id  \
0            0  cv000    29590        0   
1            0  cv000    29590        1   
2            0  cv000    29590        2   
3            0  cv000    29590        3   
4            0  cv000    29590        4   
...        ...    ...      ...      ...   
64715        9  cv999    14636       20   
64716        9  cv999    14636       21   
64717        9  cv999    14636       22   
64718        9  cv999    14636       23   
64719        9  cv999    14636       24   

                                                    text  tag  
0      films adapted from comic books have had plenty...  pos  
1      for starters  it was created by alan moore  an...  pos  
2      to say moore and campbell thoroughly researche...  pos  
3      the book  or  graphic novel   if you will  is ...  pos  
4      in other words  dont dismiss this film because...  pos  
...                                                  ...  ...  
64715  that lack of inspiration can

In [51]:
data["text"] = data["text"].str.lower()
data

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,for starters it was created by alan moore an...,pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,the book or graphic novel if you will is ...,pos
4,0,cv000,29590,4,in other words dont dismiss this film because...,pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,after watching one of the roxbury skits on s...,neg
64718,9,cv999,14636,23,bump unsuspecting women and thats all,neg


In [52]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# Tokenization: splitting the string into a list of words
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
#used a lambda expression :
data['text'] = data['text'].apply(lambda x: word_tokenize(str(x)))

print(data['text'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0        [films, adapted, from, comic, books, have, had...
1        [for, starters, it, was, created, by, alan, mo...
2        [to, say, moore, and, campbell, thoroughly, re...
3        [the, book, or, graphic, novel, if, you, will,...
4        [in, other, words, dont, dismiss, this, film, ...
                               ...                        
64715    [that, lack, of, inspiration, can, be, traced,...
64716    [like, too, many, of, the, skits, on, the, cur...
64717    [after, watching, one, of, the, roxbury, skits...
64718         [bump, unsuspecting, women, and, thats, all]
64719    [after, watching, anightattheroxbury, youll, b...
Name: text, Length: 64720, dtype: object


In [53]:
#Removing stop_words : Removing not meaningful words.
from nltk.corpus import stopwords

nltk.download('stopwords')

# Get the English stop words

stop_words = stopwords.words('english')


data['text'] = data['text'].apply(lambda sentence_tokens: [word for word in sentence_tokens if word not in stop_words ])

print(data['text'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [films, adapted, comic, books, plenty, success...
1        [starters, created, alan, moore, eddie, campbe...
2        [say, moore, campbell, thoroughly, researched,...
3        [book, graphic, novel, 500, pages, long, inclu...
4                     [words, dont, dismiss, film, source]
                               ...                        
64715    [lack, inspiration, traced, back, insipid, cha...
64716    [like, many, skits, current, incarnation, satu...
64717    [watching, one, roxbury, skits, snl, come, awa...
64718                   [bump, unsuspecting, women, thats]
64719    [watching, anightattheroxbury, youll, left, ex...
Name: text, Length: 64720, dtype: object


In [54]:
#word2vec :
import gensim
#affecting to each word of my dataSet a vector (size=100)

# training my Word2Vec model
model =gensim.models.Word2Vec(sentences=data['text'], vector_size=100, window=5, min_count=1, workers=8)

In [55]:
#we can use the model to calculate the similariy between two words
model.wv.similarity(w1="medium", w2="mid")

0.9858998

In [56]:
# for example the vector representing the word films is : 
word_vector = model.wv['medium']
word_vector 

array([-0.11371017,  0.23658514,  0.11907139,  0.02680179,  0.07633563,
       -0.37607774,  0.07109237,  0.35787138, -0.1033361 , -0.14891931,
       -0.04476611, -0.25191915, -0.10972837,  0.08973026,  0.1589317 ,
       -0.1829076 ,  0.03247021, -0.23724377,  0.01456126, -0.37496755,
        0.11318682,  0.1635985 ,  0.1546356 , -0.10595743, -0.05694155,
        0.00347008, -0.16999654,  0.02026771, -0.31001177,  0.02527394,
        0.13495469, -0.05862148,  0.03337866, -0.10819975,  0.03973536,
        0.14785743, -0.03083589, -0.13162921, -0.04839679, -0.34127417,
       -0.00756371, -0.22823255, -0.11172352, -0.0401496 ,  0.11674498,
       -0.01388765, -0.1660169 ,  0.01049154,  0.08583508,  0.23191142,
        0.06678457, -0.15054525, -0.08441942, -0.03786461, -0.07992531,
        0.16246246,  0.1449552 ,  0.04094772, -0.19906922,  0.10632405,
        0.050855  ,  0.04764174, -0.17746527,  0.06467976, -0.12429555,
        0.12130193,  0.01522932,  0.07646833, -0.20834838,  0.22

In [57]:
import numpy as np
# pour chaque mot du vocabulaire , on va récuperer le vecteur correspondants et on va  stocké l'ensemble de ces vecteurs dans un tab
def get_word_embeddings(review, model):
    word_embeddings = []
    for word in review:
        #Vérifier Si le mot est présent dans le vocabulaire du modèle :c'est-à-dire qu'il a été vu lors de l'entraînement du modèle
        if word in model.wv.key_to_index:
            word_embeddings.append(model.wv[word])
    return word_embeddings

# Fonction pour obtenir le vecteur moyen d'une critique
def get_review_vector(review, model):
    word_embeddings = get_word_embeddings(review, model)
    if word_embeddings:
        return np.mean(word_embeddings, axis=0)
    else:
        # Si la critique ne contient pas de mots dans le vocabulaire du modèle, retourner un vecteur nul
        return np.zeros(model.vector_size)


data['review_vector'] = data['text'].apply(lambda x: get_review_vector(x, model))
data['review_vector']

0        [-0.38457566, 0.6921082, 0.26475358, 0.0576685...
1        [-0.2358836, 0.39502096, 0.2510185, 0.07911752...
2        [-0.42818612, 0.75692177, 0.2970959, 5.729935e...
3        [-0.24574526, 0.443748, 0.24293397, 0.08014274...
4        [-0.47378436, 0.8001005, 0.2753085, 0.12274642...
                               ...                        
64715    [-0.2291087, 0.5947063, 0.27173412, -0.0759562...
64716    [-0.3075606, 0.5889383, 0.22496998, -0.0131542...
64717    [-0.42597672, 0.70531964, 0.35018417, 0.050108...
64718    [-0.29888692, 0.58587724, 0.26110876, 0.064377...
64719    [-0.55378985, 0.6722865, 0.33155996, 0.2461058...
Name: review_vector, Length: 64720, dtype: object

In [66]:
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split

# Diviser les données : partie training et testing
X_train, X_test, y_train, y_test = train_test_split(data['review_vector'].values.tolist(), data['tag'], test_size=0.2, random_state=42)

# Convertir les listes de vecteurs en matrices NumPy : pour que les features et targets aient des formes compatibles
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

# Créer et entraîner le modèle de régression logistique
model1 = linear_model.LogisticRegression()
model1.fit(X_train, y_train)

# Évaluer le modèle sur l'ensemble de test
accuracy = model1.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.570920889987639


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [67]:
#prediction :(sachant que le .score l'effectue implicitement)
y_pred = model1.predict(X_test)


print("Prédictions :", y_pred)

Prédictions : ['pos' 'pos' 'pos' ... 'neg' 'neg' 'pos']


In [68]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculer les prédictions sur l'ensemble de test
y_pred = model1.predict(X_test)

# Calculer les métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Afficher les résultats
print("Accuracy :", accuracy)
print("Precision :", precision)
print("Recall :", recall)
print("F1-score :", f1)

Accuracy : 0.570920889987639
Precision : 0.5731664424140476
Recall : 0.570920889987639
F1-score : 0.5645694655052206
